In [70]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA as pca
from sklearn.preprocessing import StandardScaler as stscale
#import scipy.stats as stats
from scipy import stats, linalg

import diff_classifier.aws as aws

# Importing and Scaling Data

In [2]:
filename = 'ferret_stats.csv'
folder = 'ferret_tracking'
#aws.download_s3('{}/{}'.format(folder, filename), filename, bucket_name='ccurtis.data')

In [3]:
fstats = pd.read_csv(filename, index_col='Unnamed: 0')
fstats_raw = fstats.drop('run', axis=1).as_matrix()

In [4]:
scaler = stscale()
scaler.fit(fstats_raw)
fstats_scaled = scaler.transform(fstats_raw)

In [59]:
fstats_scaled.shape

(29, 11)

# Bartlett Test

In [73]:
fstats_list = []
for num in range(0, fstats_scaled.shape[0]):
    fstats_list.append(fstats_scaled[num, :])
    
stats.bartlett(*fstats_list)

BartlettResult(statistic=56.177828137640702, pvalue=0.0012236722112932415)

We accept the null hypothesis that all input samples are from populations with equal variances.

# Kaiser-Meyer-Olkin (KMO) Measure

* 0.00 to 0.49 unacceptable.
* 0.50 to 0.59 miserable.
* 0.60 to 0.69 mediocre.
* 0.70 to 0.79 middling.
* 0.80 to 0.89 meritorious.
* 0.90 to 1.00 marvelous.

In [101]:
#Correlation matrix and the partial covariance matrix.
corrmatrix = np.corrcoef(fstats_scaled.transpose())
pcorr = partial_corr(fstats_scaled)

#Calculation of the KMO statistic
matrix = corrmatrix*corrmatrix
rows = matrix.shape[0]
cols = matrix.shape[1]
rij = 0
uij = 0
for row in range(0, rows):
    for col in range(0, cols):
        if not row == col:
            rij = rij + matrix[row, col]
            uij = uij + pcorr[row, col]

mo = rij/(rij+uij)
print(mo)

0.738151573532


We got middling results with our sampling. Will proceed regardless.

# PCA Analysis

In [104]:
# pca1 = pca(n_components=5)
# pca1.fit(fstats_raw)

# print(pca1.explained_variance_ratio_) 
# print(pca1.singular_values_)  

In [103]:
pca1 = pca(n_components=5)
pca1.fit(fstats_scaled)

print('Largest eigenvalues of covariance matrix: {}'.format(pca1.explained_variance_))
print('Percent explained variance: {}'.format(pca1.explained_variance_ratio_)) 
#print(pca1.singular_values_)  

Largest eigenvalues of covariance matrix: [ 5.24011107  2.60729756  1.72109247  0.95054192  0.40602083]
Percent explained variance: [ 0.45994705  0.2288537   0.15106768  0.08343315  0.03563819]


We picked components that met the following criteria:

* Eigenvalues greater than 1
* Percent explained variance cutoff of 80%

This gave the first three components.

In [105]:
comps = pca1.components_
pd.DataFrame(comps.transpose())

,0,1,2,3,4
0,0.111620,0.171225,-0.366750,0.822218,0.134923
1,-0.174187,0.514050,-0.246093,-0.142895,-0.132035
2,0.377786,0.212765,0.107678,-0.359908,-0.002417
3,-0.246729,0.403124,-0.255304,-0.028232,-0.601346
4,-0.141184,0.472031,-0.207095,-0.259284,0.684251
5,-0.204005,0.258350,0.586546,0.159723,-0.000015
6,-0.173756,0.304213,0.566051,0.257486,0.085113
7,-0.396533,-0.225733,-0.083302,-0.066185,0.168371
8,-0.401594,-0.203942,-0.117153,-0.000340,0.238459
9,0.431846,0.102823,0.011552,-0.025822,0.017236


In [115]:
pd.DataFrame(pca1.transform(fstats_scaled))

,0,1,2,3,4
0,-1.823586,0.511146,-0.183260,0.603651,0.084350
1,0.714671,1.449112,0.723101,-1.719787,-0.150274
2,-2.213266,0.004228,0.640305,0.609867,0.781309
3,-2.532690,0.247979,-0.678993,-0.988659,0.385243
4,-1.231830,-0.125427,-2.480964,-0.314463,0.928405
5,-1.931440,1.102193,-0.419756,0.158031,0.222572
6,0.926730,2.262759,-0.978564,-0.768400,-0.689729
7,-0.737747,4.211894,1.102683,-1.372910,1.147718
8,0.138161,0.929948,-0.545121,-0.143017,0.586309
9,-2.658847,1.037631,-0.497696,0.711818,0.590156


In [48]:
fstats

,amplitude,deviation,period,range,rsd,cross,crossdensity,pawcount,pawdensity,run,stride,stridestd
0,28.329995,22.94,22.888148,63.97,40.0,7.0,1.404537,15.0,3.009721,N_P1_R1,35.598929,7.904682
1,6.499281,23.74,45.568532,61.58,43.0,7.0,1.276366,10.0,1.823380,N_P1_R2,60.936889,23.399999
2,24.748830,20.37,21.376220,58.90,42.0,8.0,1.528406,17.0,3.247864,N_P1_R3,32.713813,7.232158
3,10.149308,25.03,20.500036,66.08,44.0,6.0,1.236932,18.0,3.710797,N_P1_R4,28.533588,19.379037
4,28.657133,23.95,22.039320,61.14,48.0,4.0,0.737719,17.0,3.135305,N_P2_R1,33.888250,19.117453
5,25.685890,24.71,23.850932,66.03,45.0,7.0,1.377782,15.0,2.952390,N_P2_R2,36.290214,9.668581
6,25.296070,26.20,46.490463,71.50,43.0,6.0,1.060071,10.0,1.766784,N_P2_R3,62.888889,27.056436
7,11.085945,27.80,44.065898,70.84,63.0,9.0,1.876904,11.0,2.293994,N_P3_R1,47.951300,38.185976
8,24.736746,21.43,29.317955,61.83,48.0,6.0,1.154959,11.0,2.117425,N_P3_R2,51.949900,23.051705
9,32.114710,23.73,21.067034,68.03,47.0,8.0,1.371225,19.0,3.256659,N_P3_R3,32.412222,10.199793


In [71]:
def partial_corr(C):
    """
    Returns the sample linear partial correlation coefficients between pairs of variables in C, controlling 
    for the remaining variables in C.

    Partial Correlation in Python (clone of Matlab's partialcorr)

    This uses the linear regression approach to compute the partial 
    correlation (might be slow for a huge number of variables). The 
    algorithm is detailed here:

        http://en.wikipedia.org/wiki/Partial_correlation#Using_linear_regression

    Taking X and Y two variables of interest and Z the matrix with all the variable minus {X, Y},
    the algorithm can be summarized as

        1) perform a normal linear least-squares regression with X as the target and Z as the predictor
        2) calculate the residuals in Step #1
        3) perform a normal linear least-squares regression with Y as the target and Z as the predictor
        4) calculate the residuals in Step #3
        5) calculate the correlation coefficient between the residuals from Steps #2 and #4; 

        The result is the partial correlation between X and Y while controlling for the effect of Z


    Date: Nov 2014
    Author: Fabian Pedregosa-Izquierdo, f@bianp.net
    Testing: Valentina Borghesani, valentinaborghesani@gmail.com

    Parameters
    ----------
    C : array-like, shape (n, p)
        Array with the different variables. Each column of C is taken as a variable


    Returns
    -------
    P : array-like, shape (p, p)
        P[i, j] contains the partial correlation of C[:, i] and C[:, j] controlling
        for the remaining variables in C.
    """
    
    C = np.asarray(C)
    p = C.shape[1]
    P_corr = np.zeros((p, p), dtype=np.float)
    for i in range(p):
        P_corr[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=np.bool)
            idx[i] = False
            idx[j] = False
            beta_i = linalg.lstsq(C[:, idx], C[:, j])[0]
            beta_j = linalg.lstsq(C[:, idx], C[:, i])[0]

            res_j = C[:, j] - C[:, idx].dot( beta_i)
            res_i = C[:, i] - C[:, idx].dot(beta_j)
            
            corr = stats.pearsonr(res_i, res_j)[0]
            P_corr[i, j] = corr
            P_corr[j, i] = corr
        
    return P_corr